In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./images\\N1.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./images\\N1.jpeg'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['./images\\N1.jpeg',
 './images\\N100.jpeg',
 './images\\N101.jpeg',
 './images\\N102.jpeg',
 './images\\N103.jpeg',
 './images\\N104.jpeg',
 './images\\N105.jpeg',
 './images\\N106.jpeg',
 './images\\N107.jpeg',
 './images\\N108.jpeg',
 './images\\N109.jpeg',
 './images\\N11.jpeg',
 './images\\N110.jpeg',
 './images\\N111.jpeg',
 './images\\N112.jpeg',
 './images\\N113.jpeg',
 './images\\N114.jpeg',
 './images\\N116.jpeg',
 './images\\N117.jpeg',
 './images\\N118.jpeg',
 './images\\N119.jpeg',
 './images\\N12.jpeg',
 './images\\N120.jpeg',
 './images\\N121.jpeg',
 './images\\N122.jpeg',
 './images\\N123.jpeg',
 './images\\N124.jpeg',
 './images\\N126.jpeg',
 './images\\N127.jpeg',
 './images\\N128.jpeg',
 './images\\N129.jpeg',
 './images\\N130.jpeg',
 './images\\N131.jpeg',
 './images\\N132.jpeg',
 './images\\N133.jpeg',
 './images\\N134.jpeg',
 './images\\N135.jpeg',
 './images\\N136.jpeg',
 './images\\N137.jpeg',
 './images\\N138.jpeg',
 './images\\N139.jpeg',
 './images\\N14.jpeg

#### verify image and output

In [8]:
file_path = image_path[0]
file_path 

'./images\\N1.jpeg'

In [9]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# 1093	1396	645	727
cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [12]:
labels = df.iloc[:,1:].values

In [13]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [14]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [15]:
X.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [16]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

### Deep Learning Model

In [17]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [18]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [19]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

conv2d_79 (Conv2D)              (None, 12, 12, 192)  215040      activation_78[0][0]              
__________________________________________________________________________________________________
batch_normalization_76 (BatchNo (None, 12, 12, 192)  576         conv2d_76[0][0]                  
__________________________________________________________________________________________________
batch_normalization_79 (BatchNo (None, 12, 12, 192)  576         conv2d_79[0][0]                  
__________________________________________________________________________________________________
activation_76 (Activation)      (None, 12, 12, 192)  0           batch_normalization_76[0][0]     
__________________________________________________________________________________________________
activation_79 (Activation)      (None, 12, 12, 192)  0           batch_normalization_79[0][0]     
__________________________________________________________________________________________________
block17_1_

batch_normalization_112 (BatchN (None, 12, 12, 192)  576         conv2d_112[0][0]                 
__________________________________________________________________________________________________
batch_normalization_115 (BatchN (None, 12, 12, 192)  576         conv2d_115[0][0]                 
__________________________________________________________________________________________________
activation_112 (Activation)     (None, 12, 12, 192)  0           batch_normalization_112[0][0]    
__________________________________________________________________________________________________
activation_115 (Activation)     (None, 12, 12, 192)  0           batch_normalization_115[0][0]    
__________________________________________________________________________________________________
block17_10_mixed (Concatenate)  (None, 12, 12, 384)  0           activation_112[0][0]             
                                                                 activation_115[0][0]             
__________

__________________________________________________________________________________________________
conv2d_150 (Conv2D)             (None, 12, 12, 160)  143360      activation_149[0][0]             
__________________________________________________________________________________________________
batch_normalization_150 (BatchN (None, 12, 12, 160)  480         conv2d_150[0][0]                 
__________________________________________________________________________________________________
activation_150 (Activation)     (None, 12, 12, 160)  0           batch_normalization_150[0][0]    
__________________________________________________________________________________________________
conv2d_148 (Conv2D)             (None, 12, 12, 192)  208896      block17_18_ac[0][0]              
__________________________________________________________________________________________________
conv2d_151 (Conv2D)             (None, 12, 12, 192)  215040      activation_150[0][0]             
__________

block8_5_mixed (Concatenate)    (None, 5, 5, 448)    0           activation_179[0][0]             
                                                                 activation_182[0][0]             
__________________________________________________________________________________________________
block8_5_conv (Conv2D)          (None, 5, 5, 2080)   933920      block8_5_mixed[0][0]             
__________________________________________________________________________________________________
block8_5 (Lambda)               (None, 5, 5, 2080)   0           block8_4_ac[0][0]                
                                                                 block8_5_conv[0][0]              
__________________________________________________________________________________________________
block8_5_ac (Activation)        (None, 5, 5, 2080)   0           block8_5[0][0]                   
__________________________________________________________________________________________________
conv2d_184

Total params: 73,663,490
Trainable params: 19,326,754
Non-trainable params: 54,336,736
__________________________________________________________________________________________________


### model training

In [20]:
from tensorflow.keras.callbacks import TensorBoard

In [21]:
tfb = TensorBoard('object_detection')

In [22]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
 1/18 [>.............................] - ETA: 0s - loss: 0.1285WARNING:tensorflow:From C:\Users\Srikanth\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
18/18 [==============================] - 106s 6s/step - loss: 0.1078 - val_loss: 0.0965
Epoch 2/100
18/18 [==============================] - 88s 5s/step - loss: 0.0977 - val_loss: 0.0802
Epoch 3/100
18/18 [==============================] - 56s 3s/step - loss: 0.0967 - val_loss: 0.0894
Epoch 4/100
18/18 [==============================] - 55s 3s/step - loss: 0.0945 - val_loss: 0.0805
Epoch 5/100
18/18 [==============================] - 63s 3s/step - loss: 0.0866 - val_loss: 0.0891
Epoch 6/100
18/18 [==============================] - 61s 3s/step - loss: 0.0824 - val_loss: 0.0746
Epoch 7/100
18/18 [==============================] - 59s 

18/18 [==============================] - 56s 3s/step - loss: 0.0696 - val_loss: 0.0665
Epoch 80/100
18/18 [==============================] - 2665s 148s/step - loss: 0.0701 - val_loss: 0.0678
Epoch 81/100
18/18 [==============================] - 142s 8s/step - loss: 0.0708 - val_loss: 0.0674
Epoch 82/100
18/18 [==============================] - 18s 1s/step - loss: 0.0705 - val_loss: 0.0657
Epoch 83/100
18/18 [==============================] - 19s 1s/step - loss: 0.0718 - val_loss: 0.0689
Epoch 84/100
18/18 [==============================] - 19s 1s/step - loss: 0.0739 - val_loss: 0.0688
Epoch 85/100
18/18 [==============================] - 18s 1s/step - loss: 0.0719 - val_loss: 0.0664
Epoch 86/100
18/18 [==============================] - 18s 1s/step - loss: 0.0677 - val_loss: 0.0638
Epoch 87/100
18/18 [==============================] - 19s 1s/step - loss: 0.0691 - val_loss: 0.0581
Epoch 88/100
18/18 [==============================] - 19s 1s/step - loss: 0.0580 - val_loss: 0.0411
Epoch 89

In [23]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
18/18 [==============================] - 34s 2s/step - loss: 2.9122e-04 - val_loss: 0.0063
Epoch 103/200
18/18 [==============================] - 21s 1s/step - loss: 1.9947e-04 - val_loss: 0.0059
Epoch 104/200
18/18 [==============================] - 21s 1s/step - loss: 2.0441e-04 - val_loss: 0.0061
Epoch 105/200
18/18 [==============================] - 19s 1s/step - loss: 1.8242e-04 - val_loss: 0.0059
Epoch 106/200
18/18 [==============================] - 19s 1s/step - loss: 1.5497e-04 - val_loss: 0.0059
Epoch 107/200
18/18 [==============================] - 21s 1s/step - loss: 2.8745e-04 - val_loss: 0.0060
Epoch 108/200
18/18 [==============================] - 20s 1s/step - loss: 3.3103e-04 - val_loss: 0.0063
Epoch 109/200
18/18 [==============================] - 21s 1s/step - loss: 2.3359e-04 - val_loss: 0.0059
Epoch 110/200
18/18 [==============================] - 22s 1s/step - loss: 1.7057e-04 - val_loss: 0.0061
Epoch 111/200
18/18 [==============================] - 

18/18 [==============================] - 20s 1s/step - loss: 4.3210e-04 - val_loss: 0.0059
Epoch 178/200
18/18 [==============================] - 20s 1s/step - loss: 5.2341e-04 - val_loss: 0.0058
Epoch 179/200
18/18 [==============================] - 20s 1s/step - loss: 5.5556e-04 - val_loss: 0.0063
Epoch 180/200
18/18 [==============================] - 23s 1s/step - loss: 4.1865e-04 - val_loss: 0.0065
Epoch 181/200
18/18 [==============================] - 21s 1s/step - loss: 4.0438e-04 - val_loss: 0.0064
Epoch 182/200
18/18 [==============================] - 20s 1s/step - loss: 4.0499e-04 - val_loss: 0.0065
Epoch 183/200
18/18 [==============================] - 20s 1s/step - loss: 4.8212e-04 - val_loss: 0.0057
Epoch 184/200
18/18 [==============================] - 20s 1s/step - loss: 5.5655e-04 - val_loss: 0.0065
Epoch 185/200
18/18 [==============================] - 20s 1s/step - loss: 6.0796e-04 - val_loss: 0.0065
Epoch 186/200
18/18 [==============================] - 20s 1s/step - 

In [25]:
model.save('./models/object_detection.h5')